# Expense Prediction Model Training (Kaggle Dataset)

This notebook trains a Machine Learning model to predict next month's expenses using the **Personal Budget Transactions Dataset**.

## 1. Setup and Data Loading

1.  Download the dataset from [Kaggle](https://www.kaggle.com/datasets/ismetsemedov/personal-budget-transactions-dataset/data).
2.  Upload `personal_budget_transactions.csv` (or whatever the file is named) to the Colab file storage (folder icon on the left).

In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import joblib

# Load the dataset
# Make sure the filename matches what you uploaded
filename = 'personal_budget_transactions.csv'

try:
    df_raw = pd.read_csv(filename)
    print("Dataset loaded successfully.")
    print(df_raw.head())
except FileNotFoundError:
    print(f"Error: {filename} not found. Please upload the CSV file.")
    # Fallback for testing if file not present
    df_raw = pd.DataFrame(columns=['date', 'amount', 'category'])

Dataset loaded successfully.
                        date    category  amount
0  2022-07-06 05:57:10 +0000  Restuarant    5.50
1  2022-07-06 05:57:27 +0000      Market    2.00
2  2022-07-06 05:58:12 +0000       Coffe   30.10
3  2022-07-06 05:58:25 +0000      Market   17.33
4  2022-07-06 05:59:00 +0000  Restuarant    5.50


## 2. Preprocessing
We need to:
1.  Normalize column names.
2.  Convert `date` to datetime.
3.  Aggregate by **Month** to get monthly total expenses.
4.  Create a sliding window dataset (Input: Last 6 months -> Output: Next month).

In [14]:
if not df_raw.empty:
    df_expenses = df_raw.copy()
    
    # Normalize columns to lowercase just in case
    df_expenses.columns = df_expenses.columns.str.lower().str.strip()
    
    # 2. Convert Date
    if 'date' in df_expenses.columns:
        df_expenses['date'] = pd.to_datetime(df_expenses['date'])
        
        # 3. Aggregate Monthly
        # Group by Year-Month
        monthly_expenses = df_expenses.set_index('date').resample('M')['amount'].sum()
        
        print("Monthly Expenses:")
        print(monthly_expenses.head())

        # 4. Create Sliding Window Dataset
        window_size = 6
        data = []
        targets = []

        # We need at least window_size + 1 months of data
        values = monthly_expenses.values
        if len(values) > window_size:
            for i in range(len(values) - window_size):
                data.append(values[i:i+window_size])
                targets.append(values[i+window_size])
                
            X = np.array(data)
            y = np.array(targets)
            
            # Create DataFrame for visualization
            df_processed = pd.DataFrame(X, columns=[f'month_{i+1}' for i in range(window_size)])
            df_processed['target'] = y
            print("\nProcessed Training Data:")
            print(df_processed.head())
        else:
            print("Not enough data to create training sequences. Generating dummy data for demonstration.")
            # Fallback dummy generation
            X = np.random.normal(loc=50000, scale=10000, size=(1000, 6))
            y = np.mean(X, axis=1) + np.random.normal(loc=0, scale=2000, size=1000)
    else:
        print("Error: 'date' column not found.")
        X = np.random.normal(loc=50000, scale=10000, size=(1000, 6))
        y = np.mean(X, axis=1) + np.random.normal(loc=0, scale=2000, size=1000)
else:
    print("DataFrame empty. Generating dummy data.")
    X = np.random.normal(loc=50000, scale=10000, size=(1000, 6))
    y = np.mean(X, axis=1) + np.random.normal(loc=0, scale=2000, size=1000)

Monthly Expenses:
date
2022-07-31 00:00:00+00:00    2861.38
2022-08-31 00:00:00+00:00    1307.52
2022-09-30 00:00:00+00:00    1338.48
2022-10-31 00:00:00+00:00    2001.12
2022-11-30 00:00:00+00:00    1017.81
Freq: ME, Name: amount, dtype: float64

Processed Training Data:
   month_1  month_2  month_3  month_4  month_5  month_6   target
0  2861.38  1307.52  1338.48  2001.12  1017.81  1605.53  2162.19
1  1307.52  1338.48  2001.12  1017.81  1605.53  2162.19  1540.84
2  1338.48  2001.12  1017.81  1605.53  2162.19  1540.84  2274.61
3  2001.12  1017.81  1605.53  2162.19  1540.84  2274.61  2224.88
4  1017.81  1605.53  2162.19  1540.84  2274.61  2224.88  2599.92


C:\Users\BIMSARA\AppData\Local\Temp\ipykernel_5636\3576288695.py:13: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_expenses = df_expenses.set_index('date').resample('M')['amount'].sum()


## 3. Train Model

In [15]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training samples: {X_train.shape[0]}")
print(f"Testing samples: {X_test.shape[0]}")

# Initialize Model
# Linear Regression might be underfitting if the pattern is complex.
# Let's try Random Forest.
# model = LinearRegression()
model = RandomForestRegressor(n_estimators=100, random_state=42)

# Train
model.fit(X_train, y_train)

# Evaluate
score = model.score(X_test, y_test)
print(f"Model R-squared Accuracy: {score:.4f}")

if score < 0:
    print("Warning: Negative R-squared means the model is performing poorly.")
    print("This is likely due to the small size of the dataset or lack of clear patterns.")

Training samples: 16
Testing samples: 5
Model R-squared Accuracy: -9.0005
This is likely due to the small size of the dataset or lack of clear patterns.


## 4. Save Model
Download the `expense_model.pkl` file and place it in your project's `python/` directory.

In [ ]:
joblib.dump(model, 'expense_model.pkl')
print("Model saved as expense_model.pkl")

try:
    from google.colab import files
    files.download('expense_model.pkl')
except ImportError:
    print("Not running in Colab, check local directory for file.")

In [20]:
pip install prophet



   ---------------------------------------- 0.0/12.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.1 MB ? eta -:--:--
    --------------------------------------- 0.3/12.1 MB ? eta -:--:--
   - -------------------------------------- 0.5/12.1 MB 818.4 kB/s eta 0:00:15
   -- ------------------------------------- 0.8/12.1 MB 835.7 kB/s eta 0:00:14
   --- ------------------------------------ 1.0/12.1 MB 942.9 kB/s eta 0:00:12
   --- ------------------------------------ 1.0/12.1 MB 942.9 kB/s eta 0:00:12
   ---- ----------------------------------- 1.3/12.1 MB 935.4 kB/s eta 0:00:12
   ------ --------------------------------- 1.8/12.1 MB 1.1 MB/s eta 0:00:10
   ------ --------------------------------- 2.1/12.1 MB 1.2 MB/s eta 0:00:09
   -------- ------------------------------- 2.6/12.1 MB 1.3 MB/s eta 0:00:08
   --------- ------------------------------ 2.9/12.1 MB 1.3 MB/s eta 0:00:07
   ----------- ---------------------------- 3.4/12.1 MB 1.4 MB/s eta 0:00:07
   ------

In [3]:
import pandas as pd
from prophet import Prophet
import joblib

# ===============================
# CONFIG
# ===============================
CSV_PATH = "personal_finance_tracker_dataset.csv"
MODEL_PATH = "expense_model_prophet.pkl"
FORECAST_MONTHS = 3
DATE_COL = "date"
AMOUNT_COL = "monthly_expense_total"
CURRENCY = "LKR"

# ===============================
# LOAD DATA
# ===============================
df = pd.read_csv(CSV_PATH)
print("Dataset loaded")
print("Columns in CSV:", df.columns.tolist())

# Keep only date + amount
df = df[[DATE_COL, AMOUNT_COL]].copy()

# Convert types
df[DATE_COL] = pd.to_datetime(df[DATE_COL], errors="coerce")
df[AMOUNT_COL] = pd.to_numeric(df[AMOUNT_COL], errors="coerce")
df = df.dropna(subset=[DATE_COL, AMOUNT_COL])

# ===============================
# CHECK FOR NON-ZERO DATA
# ===============================
if df[AMOUNT_COL].sum() == 0:
    raise ValueError(f"All values in '{AMOUNT_COL}' are zero. Cannot build a forecasting model.")

# ===============================
# AGGREGATE MONTHLY TOTALS
# ===============================
df["month"] = df[DATE_COL].dt.to_period("M")
monthly = df.groupby("month")[AMOUNT_COL].sum().reset_index()
monthly["month"] = monthly["month"].dt.to_timestamp()
monthly = monthly.sort_values("month")

# Fill missing months correctly without overwriting existing totals
monthly = monthly.set_index("month")
all_months = pd.date_range(start=monthly.index.min(), end=monthly.index.max(), freq='ME')  # use 'ME' for month-end
monthly = monthly.reindex(all_months, fill_value=0).rename_axis('month').reset_index()

print("\nMonthly totals:")
print(monthly.head(10))

# ===============================
# PREP DATA FOR PROPHET
# ===============================
prophet_df = monthly.rename(columns={"month": "ds", AMOUNT_COL: "y"})

# ===============================
# TRAIN MODEL
# ===============================
model = Prophet(yearly_seasonality=True, daily_seasonality=False, weekly_seasonality=False)
model.fit(prophet_df)

# ===============================
# FORECAST FUTURE
# ===============================
future = model.make_future_dataframe(periods=FORECAST_MONTHS, freq='ME')
forecast = model.predict(future)

# Show last few predictions
forecast_result = forecast[["ds", "yhat", "yhat_lower", "yhat_upper"]].tail(FORECAST_MONTHS)
print(f"\nForecast for next {FORECAST_MONTHS} months:")
print(forecast_result)

# ===============================
# SAVE MODEL
# ===============================
bundle = {
    "model": model,
    "currency": CURRENCY,
    "model_type": "Prophet"
}
joblib.dump(bundle, MODEL_PATH)
print(f"\nModel saved as {MODEL_PATH}")


Dataset loaded
Columns in CSV: ['date', 'user_id', 'monthly_income', 'monthly_expense_total', 'savings_rate', 'budget_goal', 'financial_scenario', 'credit_score', 'debt_to_income_ratio', 'loan_payment', 'investment_amount', 'subscription_services', 'emergency_fund', 'transaction_count', 'fraud_flag', 'discretionary_spending', 'essential_spending', 'income_type', 'rent_or_mortgage', 'category', 'cash_flow_status', 'financial_advice_score', 'financial_stress_level', 'actual_savings', 'savings_goal_met']

Monthly totals:
       month  monthly_expense_total
0 2019-01-31                    0.0
1 2019-02-28                    0.0
2 2019-03-31                    0.0
3 2019-04-30                    0.0
4 2019-05-31                    0.0
5 2019-06-30                    0.0
6 2019-07-31                    0.0
7 2019-08-31                    0.0
8 2019-09-30                    0.0
9 2019-10-31                    0.0

Forecast for next 3 months:
           ds  yhat    yhat_lower    yhat_upper
58 

In [2]:
# Normalize column names
df.columns = df.columns.str.lower().str.strip()

# List all available columns
print("Available columns in DataFrame:", df.columns.tolist())


Available columns in DataFrame: ['date', 'monthly_expense_total', 'month']


In [1]:
import pandas as pd
from prophet import Prophet
import joblib

# ===============================
# CONFIG
# ===============================
CSV_PATH = "personal_finance_tracker_dataset.csv"
MODEL_PATH = "expense_model_prophet.pkl"
FORECAST_MONTHS = 3
DATE_COL = "date"
AMOUNT_COL = "monthly_expense_total"
CURRENCY = "LKR"

# ===============================
# LOAD DATA
# ===============================
df = pd.read_csv(CSV_PATH)
print("Dataset loaded")
print("Columns in CSV:", df.columns.tolist())

# Normalize column names
df.columns = df.columns.str.lower().str.strip()

# Keep only date + amount
if DATE_COL not in df.columns or AMOUNT_COL not in df.columns:
    raise ValueError(f"Required columns not found: '{DATE_COL}' and '{AMOUNT_COL}'")

df = df[[DATE_COL, AMOUNT_COL]].copy()

# Convert types
df[DATE_COL] = pd.to_datetime(df[DATE_COL], errors="coerce")
df[AMOUNT_COL] = pd.to_numeric(df[AMOUNT_COL], errors="coerce")
df = df.dropna(subset=[DATE_COL, AMOUNT_COL])

# ===============================
# CHECK FOR NON-ZERO DATA
# ===============================
if df[AMOUNT_COL].sum() == 0:
    raise ValueError(
        f"All values in '{AMOUNT_COL}' are zero. Cannot build a forecasting model. "
        "Please check your CSV for actual expense data."
    )

# ===============================
# AGGREGATE MONTHLY TOTALS
# ===============================
df["month"] = df[DATE_COL].dt.to_period("M")
monthly = df.groupby("month")[AMOUNT_COL].sum().reset_index()
monthly["month"] = monthly["month"].dt.to_timestamp()
monthly = monthly.sort_values("month")

# Fill missing months without overwriting existing totals
monthly = monthly.set_index("month")
all_months = pd.date_range(start=monthly.index.min(), end=monthly.index.max(), freq='ME')
monthly = monthly.reindex(all_months, fill_value=0).rename_axis('month').reset_index()

print("\nMonthly totals:")
print(monthly.head(10))

# ===============================
# PREP DATA FOR PROPHET
# ===============================
prophet_df = monthly.rename(columns={"month": "ds", AMOUNT_COL: "y"})

# ===============================
# TRAIN MODEL
# ===============================
model = Prophet(yearly_seasonality=True, daily_seasonality=False, weekly_seasonality=False)
model.fit(prophet_df)

# ===============================
# FORECAST FUTURE
# ===============================
future = model.make_future_dataframe(periods=FORECAST_MONTHS, freq='ME')
forecast = model.predict(future)

# Show last few predictions
forecast_result = forecast[["ds", "yhat", "yhat_lower", "yhat_upper"]].tail(FORECAST_MONTHS)
print(f"\nForecast for next {FORECAST_MONTHS} months:")
print(forecast_result)

# ===============================
# SAVE MODEL
# ===============================
bundle = {
    "model": model,
    "currency": CURRENCY,
    "model_type": "Prophet"
}
joblib.dump(bundle, MODEL_PATH)
print(f"\nModel saved as {MODEL_PATH}")


Dataset loaded
Columns in CSV: ['date', 'user_id', 'monthly_income', 'monthly_expense_total', 'savings_rate', 'budget_goal', 'financial_scenario', 'credit_score', 'debt_to_income_ratio', 'loan_payment', 'investment_amount', 'subscription_services', 'emergency_fund', 'transaction_count', 'fraud_flag', 'discretionary_spending', 'essential_spending', 'income_type', 'rent_or_mortgage', 'category', 'cash_flow_status', 'financial_advice_score', 'financial_stress_level', 'actual_savings', 'savings_goal_met']

Monthly totals:
       month  monthly_expense_total
0 2019-01-31                    0.0
1 2019-02-28                    0.0
2 2019-03-31                    0.0
3 2019-04-30                    0.0
4 2019-05-31                    0.0
5 2019-06-30                    0.0
6 2019-07-31                    0.0
7 2019-08-31                    0.0
8 2019-09-30                    0.0
9 2019-10-31                    0.0

Forecast for next 3 months:
           ds  yhat    yhat_lower    yhat_upper
58 

In [3]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

# 1. Load the dataset
df = pd.read_csv('personal_finance_tracker_dataset.csv')

# 2. Data Alignment
df['monthly_expense_total'] = df['essential_spending'] + df['discretionary_spending']

# 3. Select Features and Target
features = ['essential_spending', 'discretionary_spending', 'rent_or_mortgage', 'loan_payment', 'subscription_services']
target = 'monthly_expense_total'

X = df[features]
y = df[target]

# 4. Train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = RandomForestRegressor(n_estimators=100, oob_score=True, random_state=42)
model.fit(X_train, y_train)

# 5. Save
joblib.dump(model, 'expense_model.pkl')
print("Model saved successfully.")

Model saved successfully.


In [9]:
import pandas as pd
import numpy as np
import joblib
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score, mean_absolute_percentage_error

# 1. Configuration & Currency Conversion (1 INR = 3.44 LKR)
EXCHANGE_RATE = 3.44  
df = pd.read_csv('monthly_spending_dataset_2020_2025.csv')
total_col_inr = 'Total Expenditure (₹)'

# Convert entire history to LKR
df['Total_Expenditure_LKR'] = df[total_col_inr] * EXCHANGE_RATE

# 2. Feature Engineering (Category-Agnostic)
# We use the previous 3 months to predict the next month
df['Month_1_Ago'] = df['Total_Expenditure_LKR'].shift(1)
df['Month_2_Ago'] = df['Total_Expenditure_LKR'].shift(2)
df['Month_3_Ago'] = df['Total_Expenditure_LKR'].shift(3)
df['Target'] = df['Total_Expenditure_LKR'].shift(-1)

# Clean data
df_final = df.dropna(subset=['Month_1_Ago', 'Month_2_Ago', 'Month_3_Ago', 'Target'])

# 3. Features and Target
features = ['Month_1_Ago', 'Month_2_Ago', 'Month_3_Ago']
X = df_final[features]
y = df_final['Target']

# 4. Split data (using the last 20% of months for testing accuracy)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# 5. Train Model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 6. Calculate Accuracy Metrics
y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)
accuracy_pct = (1 - mape) * 100

print(f"--- Model Performance (LKR) ---")
print(f"Mean Absolute Error: LKR {mae:,.2f}")
print(f"Model Accuracy Score: {accuracy_pct:.2f}%")

# 7. Save the model
joblib.dump(model, 'lkr_expense_model.pkl')
print("\nModel saved as 'lkr_expense_model.pkl'")

--- Model Performance (LKR) ---
Mean Absolute Error: LKR 13,853.70
Model Accuracy Score: 90.14%

Model saved as 'lkr_expense_model.pkl'


In [11]:
import joblib
import pandas as pd

# Load the model
model = joblib.load('lkr_expense_model.pkl')

# Provide last 3 months totals in LKR (Latest month first)
# Format: [Month_1_Ago, Month_2_Ago, Month_3_Ago]
input_data = pd.DataFrame([[145000, 138000, 150000]], 
                          columns=['Month_1_Ago', 'Month_2_Ago', 'Month_3_Ago'])

prediction = model.predict(input_data)
print(f"Predicted Total for Next Month: LKR {prediction[0]:,.2f}")

Predicted Total for Next Month: LKR 159,984.84
